# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [3]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

You should consider upgrading via the 'C:\Users\jbtok\Desktop\studia_szkola\studia\semestr8\Kryptografia\labs\venv\Scripts\python.exe -m pip install --upgrade pip' command.


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

1. Funkcja haszująca pozwala na sprawdzenie, że dana wiadomość po przehaszowaniu wciąż da odpowiedni wynik - co pozwala na zapewnienie integrity. Jeśli dojdzie do zmiany wiadomości, zmieni sie także wynik funkcji haszującej. Nie zapewnia autentykacji ani Non-repudiation - nie istnieje mechanizm pozwalający na weryfikację wysyłającego (taka sama wiadomosć wysłana przez róże osoby, da ten sam hasz) ani zapewniający, że obaj uczestnicy komunikacji dokonali podpisu (każdy może zarówno wysłać wiadomość, jak i ją zweryfikować).
2. Kod MAC pozwala na sprawdzenie integrity - posiadacz klucza może dokonać ponownego utworzenia kodu MAC na podstawie wiadomości i sprawdzić, czy doszło do zmian. Zapewnia również autentykację przez wykorzystanie tajnego klucza - tylko osoby, które go posiadają mogą utworzyć kod MAC. Nie zapewnia natomiast non-repudiation - użycie wyłącznie klucza prywatnego nie rozróżnia komunikujących się.
3. Podpis cyfrowy zapewnia integrity ponieważ pozwala na sprawdzenie, czy doszło do zmiany wiadomości, a także autentykację, bo nadawca tworzy podpis tajnym kluczem, a odbiorca ma do dyspozycji klucz publiczny. Podział kluczy pozwala także na non-repudiation - podpis utworzony przy użyciu klucza prywatnego z pary będzie odpowiadał kluczowi publicznemu, a oba są generowane przez tę samą stronę.

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [4]:
# your code here
nlen = 2048

def get_p_q_primes():
    cond_5 = False
    cond_4 = False
    cond_3 = False
    while (cond_3== False) or (cond_4 == False) or (cond_5 == False):
        p = cu.getPrime((nlen)//2)
        q = cu.getPrime((nlen)//2)
        q_same = q == p
        while q_same:
            q = cu.getPrime(nlen//2 -1)
            q_same = q == p
        if pow(2, (nlen - 1) // 2) < p < pow(2, (nlen) // 2):
            cond_3 = True
        if pow(2, (nlen - 1) // 2) < q < pow(2, (nlen) // 2):
            cond_4 = True
        if abs(p-q) > pow(2, (nlen/2)-100):
            cond_5 = True
    return p, q

def get_ned():
    cond_1 = False
    cond_2 = False
    while (cond_1== False) or (cond_2 == False):
        p,q = get_p_q_primes()
        n = p*q
        f = (p-1)*(q-1)
        e = cu.getRandomInteger(255)
        while math.gcd(f, e) != 1 and (e < f) and (e > 1):
            e = cu.getRandomInteger(255)
        if 65537 <= e < pow(2, 256):
            cond_1 = True
        if math.lcm(p-1, q-1) >= e * pow(2, nlen//2):
            cond_2 = True
    d = cu.inverse(e, f)
    return n, e, d


In [5]:
n,e,d = get_ned()
print(n,e,d)

18215259705928079982913256578173332268081255138713887895126022963655664296326863809006949002689920014043061241829348383911070288574769665681540389650987006808941565563757947061454045638688573026732288765364271192952120192895559058081981247234761558025625342548712603958448740975586581357797208630782517657619940146128897293045004631364560786847962283825919589728572447832945502510895428398648781772191561639500782081301125379906714462730456853644949176856794206472717063215914986268057490975505814348896956489644084752523153033903891371197992758481567040658343861742701934084165537451840585468514425853837066282018823 25489830826990041704899064645937917923311627492831472722345844826914054535083 4765862562099718028398384971083963452022293405988297432440396490850597818314829506605568725263605009506862678551817725958944619044751525152614338963473789447260406004464033524426637859038995803494037647501728811755703677275969487432301218217563076647226893930576977448742986052324517251757988077958072556

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [6]:
def encode(e, n, M):
    m = cu.bytes_to_long(bytes(M, "ascii"))
    C = pow(m,e,n)
    return C

def decode(d, n, C):
    m = pow(C, d, n)
    length = math.ceil(m.bit_length() / 8)
    return cu.long_to_bytes(m, length)

In [7]:
M = "Ala ma kota"
print(n, e, d)

18215259705928079982913256578173332268081255138713887895126022963655664296326863809006949002689920014043061241829348383911070288574769665681540389650987006808941565563757947061454045638688573026732288765364271192952120192895559058081981247234761558025625342548712603958448740975586581357797208630782517657619940146128897293045004631364560786847962283825919589728572447832945502510895428398648781772191561639500782081301125379906714462730456853644949176856794206472717063215914986268057490975505814348896956489644084752523153033903891371197992758481567040658343861742701934084165537451840585468514425853837066282018823 25489830826990041704899064645937917923311627492831472722345844826914054535083 4765862562099718028398384971083963452022293405988297432440396490850597818314829506605568725263605009506862678551817725958944619044751525152614338963473789447260406004464033524426637859038995803494037647501728811755703677275969487432301218217563076647226893930576977448742986052324517251757988077958072556

In [8]:
C = encode(e, n, M)
print(C)

16065795379733888948177478166638275578649919155989809183605754395986506692562485832404088048555085591684008117785400329286516869021027829583556292949709533862370971876736085502971136272024541585329691953960456510095304699435706235752866618404754337102416563979066708006549128428465856564911667919946219498425062237767600594054867223883261743097659849594013907589355358544145501043720442642161629944692979370612989472232982692317629716462460750009836048712463064891091564970431419347938411354113729364942941767891186431903923171664666788469779333425850076226138859415541404636240320680019593159142700980792434672012800


In [9]:
m = decode(d, n, C)
print(m)

b'Ala ma kota'


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [10]:
from Crypto.Hash import SHA256

def H(M):
    h = SHA256.new(M)
    return h.digest()

def get_signature(M, d, n):
    h = H(M)
    h_n = cu.bytes_to_long(h)
    SIG = pow(h_n, d, n)
    return SIG

def verify_signature(M,SIG, e, n):
    h = H(M)
    h_n = cu.bytes_to_long(h)
    VER = pow(SIG, e, n)
    if VER == h_n:
        return True
    else:
        return False

In [11]:
mes = 'Ala ma kota'
print(H(bytes(mes, "ascii")))

b'\x12K\xfbb\x84\xd8/;\x11\x05\xf8\x8e>z\x0e\xe0-\x0eRQ\x93A<\x05\xb7PA\x91p"\xcdn'


In [12]:
sig = get_signature(bytes(mes, "ascii"), d, n)
print(sig)

7176255022475825054613482488439075686779074747535751484307695791566355991588460435769649229849579395935195104682228039634550278649863614327749311852677349046214153705300557775305023635914656748592725876401658573925224881412551818637778157243271781145117927319991746020008488918798923473107252332769339181142128368980378617615203361463854213258204099961353404306226863377369461824743622260205716334064991830949498542995981153512343881715603610240934223029300453390071123344175427296294115045672336566807185631645343212815843282490299441341191381142039923623994156578883389859442181256193136184337139493295595304391432


In [13]:
print(verify_signature(bytes(mes, "ascii"),sig, e, n))

True


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [14]:
# Your code here
hlen = 256//8
def MGF1(M, len, H):
    #m_bytes = bytes(M, "ascii")
    lim = cu.ceil_div(len, hlen)
    i = 0
    tmp = H(M + i.to_bytes(4, "big"))
    output = tmp
    for i in range(1, lim):
        tmp = H(M + i.to_bytes(4, "big"))
        output = output + tmp
    return output[:len]

In [15]:
mes = "Ala ma kota"
m_bytes = bytes(mes, "ascii")
print(MGF1(m_bytes, 100, H))

b'O\x82y,\xd5\x98\x18\xc5B7\xc4YT\xe6\xfa\x82\xdd\x08\xc8\x8d\x9b?\xb2\xe2X3\x1c\tf%d\x18\xd6,\xad\x91\x8eH\x9e\xa6\x1b\xf7m\xd4\xfcu\xf5L\x0f&\x94\x10\x1a\x89\xf0e\xcdx\xb3\xa3?\x7f\xca\xbd.M#C"\xb4B\x8d\xd3\xb1GB\xe3y5\xc9 \xd4\t`B\xfa\xa9\x9e\x0c#\xea4Hb\xe0\x02\xbd\xdeG\xd2'


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [35]:
import random

def byte_xor(bytes1, bytes2):
    array = []
    for b1, b2 in zip(bytes1, bytes2):
        array.append(b1^b2)
    return bytes(array)

def OAEP_encode(H, n, M, MGF1):
    mlen = len(M)
    k = cu.size(n) // 8
    x = k - 2*hlen - 2
    if mlen > x:
        print('Error')
        return
    lhash = H(bytes("", "ascii"))
    ps = b'\0' * (x - mlen)
    db = lhash + ps + b'\1' + M
    if len(db) != k - hlen - 1:
        print("Error")
    seed = random.randbytes(hlen)
    dbMask = MGF1(seed, k - hlen - 1, H)
    maskedDB = byte_xor(db,dbMask)
    seedMask = MGF1(maskedDB, hlen, H)
    maskedSeed = byte_xor(seed, seedMask)
    em = b'\0'+maskedSeed+maskedDB
    if len(em) != k:
        print("Error")
        return
    return em


def OAEP_decode(H, n, MGF1, EM, mlen):
    k = cu.size(n) // 8
    Y = EM[0]
    if Y != 0:
        print("Error")
        return
    maskedSeed = EM[1:hlen+1]
    maskedDB = EM[hlen+1:]
    lhash = H(bytes("", "ascii"))
    seedMask = MGF1(maskedDB, hlen, H)
    seed = byte_xor(maskedSeed, seedMask)
    db_mask = MGF1(seed, k - hlen - 1, H)
    db = byte_xor(maskedDB, db_mask)
    lhash_n = db[:hlen]
    ps = db[hlen:hlen + (k - mlen - 2*hlen - 2)]
    if db[hlen + (k - mlen - 2*hlen - 2)] != 1:
        print("Error")
        print(db[hlen + (k - mlen - 2*hlen - 2)])
        M = db[hlen + (k - mlen - 2*hlen - 2) + 1:]
        print(M)
        return
    M = db[hlen + (k - mlen - 2*hlen - 2) + 1:]
    if lhash != lhash_n:
        print("Error")
        return
    return M

def RSA_OAEP_encode(e, n, M):
    em = OAEP_encode(H, n, M, MGF1)
    m = cu.bytes_to_long(em)
    C = pow(m,e,n)
    return C

def RSA_OAEP_decode(d, n, C, mlen):
    m = pow(C, d, n)
    length = cu.size(n) // 8
    em = cu.long_to_bytes(m, length)
    M = OAEP_decode(H, n, MGF1, em, mlen)
    return M

In [36]:
mes = bytes(M, "ascii")
mlen = len(mes)
M = "Ala ma kota i psa"
EM_t = RSA_OAEP_encode(e, n, bytes(M, "ascii"))
print(EM_t)

1688076892331604154368589502559726705527707087119767574721053988884512994398377902521766707262634832006210979146261561677952249734299842018393952139792064924158971472117102899328792384906480348733504970110156492037397311209501764685578201995835677240995380256890493361240322649492222836992992508265327328446752919502144844350970914261532840197816827626130123935217732979580168240415077490881092201358573289483905840395020617347233172655792063594384542814597146985096462762089518615504329410144760343656809289039818337969667787404152614449162515682061402303188779987705120477927525659011408625469386487917220672126775


In [37]:
print(RSA_OAEP_decode(d, n, EM_t, mlen))

b'Ala ma kota i psa'


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [33]:
def EMSA_encode(H, slen, M, MGF1, emBits, emlen):
    mhash = H(M)
    if emlen < hlen + slen + 2:
        print("Error")
        return
    salt = random.randbytes(slen)
    zeros = b'\0' * 8
    m_n = zeros + mhash + salt
    mhash_n = H(m_n)
    ps = b'\0' * (emlen - slen - hlen - 2)
    db = ps + b'\1' + salt
    dbMask = MGF1(mhash_n, emlen - hlen -1, H)
    dbMasked = byte_xor(db, dbMask)
    bits = 8*emlen - emBits
    bytes_len = len(dbMasked)
    bites_len = bytes_len * 8
    dbMasked_bites = "{:08b}".format(int(dbMasked.hex(),16)).zfill(bites_len)
    dbM_cut = dbMasked_bites[bits:]
    dbM_cut = dbM_cut.rjust(len(dbMasked_bites), '0')
    dbMasked_bytes = int(dbM_cut, 2).to_bytes(bytes_len, "big")
    dbMasked_bites = "{:08b}".format(int(dbMasked_bytes.hex(),16)).zfill(bites_len)
    em = dbMasked_bytes + mhash_n + b'\xbc'
    return em

def EMSA_decode(H, slen, EM, M, MGF1, emBits, emlen):
    mhash = H(M)
    if emlen < hlen + slen + 2:
        print("Error")
        return
    if EM[-1] != int(b'\xbc'.hex(), 16):
        print("Error")
        print(EM[-1])
        print(int(b'\xbc'.hex(), 16))
        return
    ind = emlen - hlen - 1
    maskedDB = EM[:ind]
    mhash_n = EM[ind:ind+hlen]
    bits = 8*emlen - emBits
    bytes_len = len(maskedDB)
    bites_len = 8*bytes_len
    dbMasked_bites = "{:08b}".format(int(maskedDB.hex(),16)).zfill(bites_len)
    leading = int(dbMasked_bites[:bits], 2)
    if leading != 0:
        print("Error")
        return
    dbMask = MGF1(mhash_n, emlen - hlen - 1, H)
    db = byte_xor(maskedDB, dbMask)
    bytes_len = len(db)
    bites_len = bytes_len * 8
    db_bites = "{:08b}".format(int(db.hex(),16)).zfill(bites_len)
    db_cut = db_bites[bits:]
    db_cut = db_cut.rjust(len(db_bites), '0')
    db_bytes = int(db_cut, 2).to_bytes(bytes_len, "big")
    v_bits = emlen - hlen - slen - 2
    leading = int(db_cut[:bits], 2)
    poz = emlen - hlen - slen - 2
    if leading != 0 or db_bytes[poz] != 1:
        print("Error")
        print(db_bytes[poz])
        return
    salt = db_bytes[-slen:]
    zeros = b'\0' * 8
    nM = zeros + mhash + salt
    if len(nM) != 8 + hlen + slen:
        print('Error')
        return
    mhash_2 = H(nM)
    if mhash_2 != mhash_n:
        print("Error")
        return
    return True

def EMSA_get_signature(M, d, n):
    emBits = 2047
    emlen = nlen // 8
    emsa_M = EMSA_encode(H, hlen, mes, MGF1, emBits, emlen)
    h = H(M)
    h_n = cu.bytes_to_long(h)
    SIG = pow(h_n, d, n)
    return SIG

def EMSA_verify_signature(M,SIG, e, n):
    emBits = 2047
    emlen = nlen // 8
    h = H(M)
    h_n = cu.bytes_to_long(h)
    emsa_VER = pow(SIG, e, n)
    emsa_v_bytes = emsa_VER.to_bytes(emlen, 'big')
    VER = EMSA_decode(H, hlen, em, M, MGF1, emBits, emlen)
    if VER:
        return True
    else:
        return False

In [23]:
m = "Ala ma podpis"
mes = bytes(M, "ascii")
mlen = len(mes)
emBits = 2047
emlen = nlen // 8
em = EMSA_encode(H, hlen, mes, MGF1, emBits, emlen)
print(em)

b"x_\xae\xacE\x1a\xa2\xe3\x94\xdf\x07R\r0\xff^\xca\xca;\xcb\x1e\xea\x9ea\xb6\xd4Lf\xcf\xe4\x03\xca\xcc@\x9c\xa7W\xb9\xae\x07\xc6\xbd\x8bt\x98\xa9\xab%>E\xcd\xa1\x1c\x11*7\x04\x8a\x10\xc5\xa0\xf2\r\xbecP\x1d\xe1\xe3yo\xfb\xf3mo\xdc\xdc\x87\x8b\xc4\xca\xa2\x9f\xfb\x9b\xe2\xcc\xbb+\xb6%\x9b4\xf7\xcb\xfd\x82\xad\xc8/\x0e\xcb\xa9'C\xc2\xf2\xde\xca\xc2\xefW\xf1\x87\xc3\xeb\xa4\xfb7\xa3\x8f\x9b\xe1\x8e\tX6\x86\xc2\x04\t\xba\xd1\xf4\xf6*r\x9d\xff\xd1G\xf2\xd9\xeb\xe8\xcc\xef>\xb9\xd8\x00:\xe3\xd5\xec\x1e\x86\xb6\xb4D|V]\xe6\x908\xb1\xb9N\x00,\xb0\x18\xacuTt|F\xb8\xc7\x87fa\xa6t\xdb\xbf<\x02\xe8\xd9\xf1\xd8$\x12\xb2\xca H\xae\xa8\xe3\xc0o\x9c\xd7C\xa4\x07\xe5:]5]\xd5\xce\x8f;K\x85yp\x08Xt\xe1|\x8b\xf0D\x96\xb3\xfc\xccn\x05\x81AC<\x18\xa5\xd0\xf6\xf8\x99\t\x1fQ\n\x16\xf3r0\xbc"


In [24]:
print(EMSA_decode(H, hlen, em, mes, MGF1, emBits, emlen))

True


In [25]:
sig = EMSA_get_signature(mes, d, n)

In [26]:
print(EMSA_verify_signature(mes,sig,e,n))

True


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [28]:
n,e,d = get_ned()
rand = cu.getRandomInteger(nlen)
key = H(n.to_bytes(nlen//8, 'big'))
klen = len(key)
ciphered_key = RSA_OAEP_encode(e, n, key)
print(key)

b'\xc1\x06\xe0\xdc\x1et?\xd0\x1f\x0c\xeb \xfe\x99X\xd4\xe8&L\xf3\xe3\xac\x02\xfb\xdf\xf9\x8bZ\x02\xfdn%'


In [29]:
rkey = RSA_OAEP_decode(d, n, ciphered_key, klen)
print(rkey)

b'\xc1\x06\xe0\xdc\x1et?\xd0\x1f\x0c\xeb \xfe\x99X\xd4\xe8&L\xf3\xe3\xac\x02\xfb\xdf\xf9\x8bZ\x02\xfdn%'
